<a href="https://colab.research.google.com/github/GeraudBourdin/llm-scripts/blob/main/convert_context_qa_load_v2_To_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### convert context_qa.load_v2 To mistral

In [ ]:
!pip install transformers mistral_common datasets

In [38]:
from datasets import Dataset, load_dataset
import json
from typing import Dict
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from transformers import AutoTokenizer

mistralTokenizer = MistralTokenizer.v1()
hf_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", revision="pr/51")
# ## variable servant a connaître le max de tokens de notre dataset
# max_dataset_token_length = 0

def format_data_from_mistral_common(data: Dict, instruct: str = '') -> Dict:
    global max_dataset_token_length

    dataset = {}
    chat = conver_to_chat(data, instruct)

    tokenized = mistralTokenizer.encode_chat_completion(ChatCompletionRequest(messages=chat))
    dataset['chat'] = chat
    dataset['json'] = data
    dataset['text'] = tokenized.text.replace('[INST][/INST]', '').replace('▁[INST]▁▁[/INST]', '')
    dataset['length'] = len(tokenized.tokens)
    ## si jamais la taille de notre instruction dépasse la derniere
    if dataset['length'] > max_dataset_token_length :
      max_dataset_token_length = dataset['length']
    return dataset

def conver_to_chat(data: Dict, instruct: str = '') -> Dict:
    prompt = ""
    answer = ""
    print(instruct)
    if instruct != '':
        prompt += instruct.strip()
        prompt += "\n"

    if "system_prompt" in data:
        prompt += data["system_prompt"].strip()
        prompt += "\n"

    if "context" in data:
        prompt += data["context"].strip()
        prompt += "\n"

    if "question" in data:
        prompt += data["question"].strip()

    if "answer" in data:
        answer += data["answer"].strip()

    if "response" in data:
        answer += data["response"].strip()

    chat = [
      {"role": "user", "content": prompt},
      {"role": "assistant", "content": answer},
      {"role": "user", "content": ""},
    ]
    del prompt, answer
    return chat

def format_data_from_hf_tokenizer(data: Dict, instruct: str = '') -> Dict:
    global max_dataset_token_length
    global hf_tokenizer

    dataset = {}
    chat = conver_to_chat(data, instruct)

    hf_text = hf_tokenizer.apply_chat_template(chat, tokenize=False)
    hf_tokens = hf_tokenizer.apply_chat_template(chat, tokenize=True)

    tokenized = mistralTokenizer.encode_chat_completion(ChatCompletionRequest(messages=chat))
    dataset['chat'] = chat
    dataset['json'] = data
    dataset['text'] = hf_tokenizer.apply_chat_template(chat, tokenize=False).replace('[INST][/INST]', '').replace('▁[INST]▁▁[/INST]', '').replace('[INST] [/INST]', '')
    dataset['length'] = len(hf_tokenizer.apply_chat_template(chat, tokenize=True))
    ## si jamais la taille de notre instruction dépasse la derniere
    if dataset['length'] > max_dataset_token_length :
      max_dataset_token_length = dataset['length']
    del chat, hf_text, hf_tokens
    return dataset

def get_dataset_from_file(file: str, mode: str = 'hf', instruct: str = '') -> Dataset:
    global max_dataset_token_length

    ## variable servant a connaître le max de tokens de notre dataset
    max_dataset_token_length = 0

    trainDatas = []
    ## on ouvre le fichier
    with open(file, 'r') as f:
      ## on boucle sur chacune des lignes
      for line in f:
        ## on mets dans data un equivalent de json_decode()
        data = json.loads(line)

        if mode == 'hf':
          ## on passe notre array json a note fonction chargée d'encoder nos informations
          ## par huggingface chatTemplate
          formatted_data = format_data_from_hf_tokenizer(data, instruct)
        elif mode == 'mistral':
          ## on passe notre array json a note fonction chargée d'encoder nos informations
          ## par mistralcommom
          formatted_data = format_data_from_mistral_common(data, instruct)
        else:
            raise ValueError('Invalid mode, mistral ou hf (for huggingface tokenizer) only.')

        ## on pousse dans notre array les informations décodés
        trainDatas.append(formatted_data)

    ## On cree un objet Dataset (de hugging face) contenant notre array trainDatas
    train_dataset = Dataset.from_dict({'text': [item['text'] for item in trainDatas] })

    return {'dataset': train_dataset, 'max_token': max_dataset_token_length}

#file = 'train.jsonl'
file = 'test.jsonl'
instruct = "You are a nice bot."
train_dataset = get_dataset_from_file(file, 'mistral', instruct)

print(train_dataset['dataset'][0])
print(train_dataset['max_token'])

train_dataset = get_dataset_from_file(file, 'hf', instruct)

print(train_dataset['dataset'][0])
print(train_dataset['max_token'])


train_dataset = get_dataset_from_file(file)

print(train_dataset['dataset'][0])
print(train_dataset['max_token'])

You are a nice bot.
You are a nice bot.
{'text': "<s>▁[INST]▁You▁are▁a▁nice▁bot.<0x0A>Le▁texte▁décrit▁les▁avantages▁de▁la▁gamme▁de▁véhicules▁électriques▁Hyundai,▁qui▁comprend▁des▁modèles▁hybrides▁légers,▁hybrides,▁hybrides▁rechargeables,▁moteurs▁électriques▁et▁voitures▁à▁pile▁à▁hydrogène.▁Les▁clients▁peuvent▁consulter▁des▁conseillers▁connectés▁pour▁choisir▁leur▁modèle▁préféré,▁commander▁en▁ligne▁et▁utiliser▁le▁service▁Charge▁myHyundai▁pour▁recharger▁leur▁véhicule▁électrique▁partout▁en▁Europe.▁Hyundai▁offre▁également▁une▁garantie▁de▁5▁ans▁kilométrage▁illimité▁et▁une▁garantie▁de▁8▁ans▁pour▁la▁batterie▁haute▁tension▁de▁leurs▁véhicules▁hybrides,▁plug-in▁et▁électriques.▁Enfin,▁le▁service▁Cl!ck▁to▁Buy▁permet▁aux▁clients▁de▁rechercher,▁configurer▁et▁commander▁leur▁voiture▁en▁ligne.\r▁#▁Commandez▁en▁ligne▁via▁Cl!ck▁to▁Buy<0x0A>Ce▁service▁vous▁permet▁de▁simplifier▁le▁processus▁d'acquisition▁en▁recherchant▁le▁modèle▁désiré▁selon▁les▁disponibilités.▁Vous▁pouvez▁le▁configurer▁et▁commander▁votre▁vo